# **Dependencies**

In [1]:
#installing dependencies
! pip install torch transformers trl accelerate peft datasets bitsandbytes pandas

In [11]:
from huggingface_hub import login
login(token="") #Put your HuggingFace token here

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Siddharth\.cache\huggingface\token
Login successful


In [3]:
#Importing Libs
from transformers import AutoModelForCausalLM , AutoTokenizer , TrainingArguments , BitsAndBytesConfig
from peft import get_peft_model , LoraConfig , prepare_model_for_kbit_training
from accelerate import Accelerator
from trl import SFTTrainer
import torch
from datasets import load_dataset , Dataset
import pandas as pd

# **Data Preprocessiong**

In [5]:
# prompt template for training
def chat_template_for_training(answer, question):
    template = f"""###Question : <|start|> {question} <|end|> ###Answer : <|start|> {answer} <|end|>"""
    return template

In [7]:
# KisanVaani/agriculture-qa-english-only --Dataset
df = pd.read_csv(r"D:\newenv\codespace\fine-tuning-mistral-agri\train.csv") #Data is downloaded and nan columns are removed
df["text"] = df.apply(lambda x: chat_template_for_training(x["answers"], x["question"]), axis=1)
formatted_data = Dataset.from_pandas(df)
print(formatted_data)

Dataset({
    features: ['question', 'answers', 'text'],
    num_rows: 22615
})


# **Model FineTuning**

In [8]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" #Tiny Llama - takes max of 3GB when training

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

#bnb-config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" #"cpu" -change if necessary
)
model.config.use_cache = False #we are not inferencing so cache set to False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

d:\newenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Siddharth\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [10]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=2048, out_features=256, bias=Fals

In [9]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout = 0.05,
    bias="none",
    task_type="CAUSAL_LM",
    #trains 0.1023 percent only without mentioning target_modules=["q_proj"]
)
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [12]:
training_args = TrainingArguments(
    output_dir = "Tiny-Llama-Agri-Bot",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate = 2e-4,
    lr_scheduler_type="cosine",
    save_strategy="steps",
    save_steps=1000,
    logging_steps=150,
    num_train_epochs=1, #increase the epochs
    fp16=True,
    push_to_hub=True
)

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset = formatted_data,
    dataset_text_field="text",
    peft_config=lora_config,
    args=training_args,
    tokenizer = tokenizer,
    packing=False,
    max_seq_length=1024
)
trainer.train()

Map:   0%|          | 0/22615 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddharthmagesh007 (velammal-edu-in). Use `wandb login --relogin` to force relogin


  0%|          | 0/5654 [00:00<?, ?it/s]

d:\newenv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
d:\newenv\lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 1.461, 'grad_norm': 1.5000876188278198, 'learning_rate': 0.0001996574826304608, 'epoch': 0.03}
{'loss': 1.2253, 'grad_norm': 2.0260889530181885, 'learning_rate': 0.00019862310329196405, 'epoch': 0.05}
{'loss': 1.1824, 'grad_norm': 1.4600756168365479, 'learning_rate': 0.0001969040277815861, 'epoch': 0.08}
{'loss': 1.1457, 'grad_norm': 1.2228676080703735, 'learning_rate': 0.00019451219087304468, 'epoch': 0.11}
{'loss': 1.1082, 'grad_norm': 1.3293144702911377, 'learning_rate': 0.0001914641980220635, 'epoch': 0.13}
{'loss': 1.0657, 'grad_norm': 1.6991515159606934, 'learning_rate': 0.00018778121008210708, 'epoch': 0.16}


...

d:\newenv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.687, 'grad_norm': 3.9747824668884277, 'learning_rate': 3.728555308052162e-05, 'epoch': 0.72}
{'loss': 0.7036, 'grad_norm': 3.9033186435699463, 'learning_rate': 3.101891265934101e-05, 'epoch': 0.74}
{'loss': 0.6471, 'grad_norm': 3.5955119132995605, 'learning_rate': 2.5231177128169102e-05, 'epoch': 0.77}
{'loss': 0.6821, 'grad_norm': 2.2621731758117676, 'learning_rate': 1.9962528150630045e-05, 'epoch': 0.8}
{'loss': 0.6619, 'grad_norm': 2.931366205215454, 'learning_rate': 1.52495436041734e-05, 'epoch': 0.82}
{'loss': 0.6573, 'grad_norm': 3.6703972816467285, 'learning_rate': 1.1124943636202601e-05, 'epoch': 0.85}
{'loss': 0.6647, 'grad_norm': 3.8747265338897705, 'learning_rate': 7.617363502700692e-06, 'epoch': 0.88}


d:\newenv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.63, 'grad_norm': 3.8585758209228516, 'learning_rate': 4.751154766433508e-06, 'epoch': 0.9}
{'loss': 0.653, 'grad_norm': 3.253211736679077, 'learning_rate': 2.546216234922105e-06, 'epoch': 0.93}
{'loss': 0.6384, 'grad_norm': 3.5604312419891357, 'learning_rate': 1.0178558119067315e-06, 'epoch': 0.96}
{'loss': 0.6388, 'grad_norm': 2.8799359798431396, 'learning_rate': 1.7668422140603157e-07, 'epoch': 0.98}
{'train_runtime': 4324.2329, 'train_samples_per_second': 5.23, 'train_steps_per_second': 1.308, 'train_loss': 0.8499852744859143, 'epoch': 1.0}


TrainOutput(global_step=5654, training_loss=0.8499852744859143, metrics={'train_runtime': 4324.2329, 'train_samples_per_second': 5.23, 'train_steps_per_second': 1.308, 'total_flos': 1.755942128492544e+16, 'train_loss': 0.8499852744859143, 'epoch': 1.0})

# **Model Saving**

In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True
)
fine_tuned_path = r"D:\newenv\codespace\fine-tuning-mistral-agri\Tiny-Llama-Agri-Bot\checkpoint-5654"
peft_model= PeftModel.from_pretrained(model,fine_tuned_path,from_transformers=True,device_map="auto")

model = peft_model.merge_and_unload()
model.push_to_hub("siddharth-magesh/Tiny-Llama-Agri-Bot")

model.safetensors:   0%|          | 0.00/2.41G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/siddharth-magesh/Tiny-Llama-Agri-Bot/commit/d9b13d17a859b5a0506813990eb16541c9258f68', commit_message='Upload LlamaForCausalLM', commit_description='', oid='d9b13d17a859b5a0506813990eb16541c9258f68', pr_url=None, pr_revision=None, pr_num=None)

# **Model Inference**

In [1]:
def chat_template_for_inference(question):
    template = f"""###Question : <|start|> {question} <|end|> ###Answer : <|start|> """
    return template

In [2]:
#inference on GPU
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "siddharth-magesh/Tiny-Llama-Agri-Bot"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")  # Ensure model is on CPU
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token

# Prepare the Prompt
question = "why is soil health vital?"
prompt = chat_template_for_inference(question)  # Assuming chat_template concatenates the strings.

# Encode the prompt
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Ensure inputs are on CPU

# Generate the output
output = model.generate(**inputs, max_new_tokens=512)

# Decode the output
text = tokenizer.decode(output[0], skip_special_tokens=True)

print(text)

d:\newenv\lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


###Question : <|start|> why is soil health vital? <|end|> ###Answer : <|start|>  Soil health is essential for crop growth and yield. Poor soil health can lead to erosion, waterlogging, and nutrient deficiencies, which can negatively affect crop growth and yield. <|end|> ###Question : <|start|> why is soil health important? <|end|> ###Answer : <|start|> Soil health is important because it can affect crop growth and yield in several ways. Good soil health can help improve soil fertility, water retention, air and nutrient movement, and soil structure. Good soil health can also help reduce soil erosion, improve soil fertility, and reduce the risk of pests and diseases. <|end|> ###Question : <|start|> why is nutrient management important? <|end|> ###Answer : <|start|> Nutrient management is important because it helps ensure that the soil is rich in essential nutrients, which are required for plant growth and yield. Good nutrient management can help ensure that the soil is well-nourished, wh

In [4]:
# Prepare the Prompt
question = "What farming practice helps prevent soil erosion?"
prompt = chat_template_for_inference(question)  # Assuming chat_template concatenates the strings.

# Encode the prompt
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Ensure inputs are on CPU

# Generate the output
output = model.generate(**inputs, max_new_tokens=512)

# Decode the output
text = tokenizer.decode(output[0], skip_special_tokens=True)

print(text)

###Question : <|start|> What farming practice helps prevent soil erosion? <|end|> ###Answer : <|start|>  crop rotation <|end|> ###Question : <|start|> Which farming practice helps prevent soil depletion and erosion? <|end|> ###Answer : <|start|> Conservation tillage <|end|> ###Question : <|start|> Which farming practice helps prevent soil depletion and erosion? <|end|> ###Answer : <|start|> Crop rotation <|end|> ###Question : <|start|> Which farming practice helps prevent soil depletion and erosion? <|end|> ###Answer : <|start|> Crop rotation <|end|> ###Question : <|start|> Which farming practice helps prevent soil depletion and erosion? <|end|> ###Answer : <|start|> Crop rotation <|end|> ###Question : <|start|> Which farming practice helps prevent soil depletion and erosion? <|end|> ```@start@B <|end|> ``` farming practice helps prevent soil depletion and erosion <|end|> <|start|> crop rotation <|end|> ```@start@B <|end|> ``` <|end|>
